---
sidebar_label: Slack
---

# SlackToolkit

This will help you getting started with the Slack [toolkit](/docs/concepts/#toolkits). For detailed documentation of all SlackToolkit features and configurations head to the [API reference](https://api.python.langchain.com/en/latest/agent_toolkits/langchain_community.agent_toolkits.slack.toolkit.SlackToolkit.html).

## Setup

To use this toolkit, you will need to get a token as explained in the [Slack API docs](https://api.slack.com/tutorials/tracks/getting-a-token). Once you've received a SLACK_USER_TOKEN, you can input it as an environment variable below.

In [ ]:
import getpass
import os

if not os.getenv("SLACK_USER_TOKEN"):
    os.environ["SLACK_USER_TOKEN"] = getpass.getpass("Enter your Slack user token: ")

If you want to get automated tracing from runs of individual tools, you can also set your [LangSmith](https://docs.smith.langchain.com/) API key by uncommenting below:

In [ ]:
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
# os.environ["LANGSMITH_TRACING"] = "true"

### Installation

This toolkit lives in the `langchain-community` package. We will also need the Slack SDK:

In [ ]:
%pip install -qU langchain-community slack_sdk

Optionally, we can install beautifulsoup4 to assist in parsing HTML messages:

In [ ]:
%pip install -qU beautifulsoup4 # This is optional but is useful for parsing HTML messages

## Instantiation

Now we can instantiate our toolkit:

In [2]:
from langchain_community.agent_toolkits import SlackToolkit

toolkit = SlackToolkit()

## Tools

View available tools:

In [3]:
tools = toolkit.get_tools()

tools

[SlackGetChannel(client=<slack_sdk.web.client.WebClient object at 0x10ce3a4d0>),
 SlackGetMessage(client=<slack_sdk.web.client.WebClient object at 0x10ce3a0e0>),
 SlackScheduleMessage(client=<slack_sdk.web.client.WebClient object at 0x10ce3a050>),
 SlackSendMessage(client=<slack_sdk.web.client.WebClient object at 0x10ce3a020>)]

This toolkit loads:

- [SlackGetChannel](https://api.python.langchain.com/en/latest/tools/langchain_community.tools.slack.get_channel.SlackGetChannel.html)
- [SlackGetMessage](https://api.python.langchain.com/en/latest/tools/langchain_community.tools.slack.get_message.SlackGetMessage.html)
- [SlackScheduleMessage](https://api.python.langchain.com/en/latest/tools/langchain_community.tools.slack.schedule_message.SlackScheduleMessage.html)
- [SlackSendMessage](https://api.python.langchain.com/en/latest/tools/langchain_community.tools.slack.send_message.SlackSendMessage.html)

## Use within an agent

Let's equip an agent with the Slack toolkit and query for information about a channel.

In [4]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

agent_executor = create_react_agent(llm, tools)

In [6]:
example_query = "When was the #general channel created?"

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    message = event["messages"][-1]
    if message.type != "tool":  # mask sensitive information
        event["messages"][-1].pretty_print()

================================ Human Message =================================

When was the #general channel created?
================================== Ai Message ==================================
Tool Calls:
  get_channelid_name_dict (call_mINmB55OWDIkXykGXZXaL5Ar)
 Call ID: call_mINmB55OWDIkXykGXZXaL5Ar
  Args:
================================== Ai Message ==================================

The #general channel was created on Unix timestamp 1671043305, which corresponds to "Mon, 12 Dec 2022 18:41:45 GMT" in human-readable format.


## Example with AgentExecutor:

In [11]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI

In [12]:
llm = ChatOpenAI(temperature=0, model="gpt-4")
prompt = hub.pull("hwchase17/openai-tools-agent")
agent = create_openai_tools_agent(
    tools=toolkit.get_tools(),
    llm=llm,
    prompt=prompt,
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke(
    {
        "input": "Send a greeting to my coworkers in the #general channel. Note use `channel` as key of channel id, and `message` as key of content to sent in the channel."
    }
)

In [13]:
agent_executor.invoke(
    {"input": "How many channels are in the workspace? Please list out their names."}
)



> Entering new AgentExecutor chain...
I need to get the list of channels in the workspace.
Action: get_channelid_name_dict
Action Input: {}[{"id": "C052SCUP4UD", "name": "general", "created": 1681297313, "num_members": 1}, {"id": "C052VBBU4M8", "name": "test-bots", "created": 1681297343, "num_members": 2}, {"id": "C053805TNUR", "name": "random", "created": 1681297313, "num_members": 2}]I now have the list of channels and their names.
Final Answer: There are 3 channels in the workspace. Their names are "general", "test-bots", and "random".

> Finished chain.


{'input': 'How many channels are in the workspace? Please list out their names.',
 'output': 'There are 3 channels in the workspace. Their names are "general", "test-bots", and "random".'}

In [14]:
agent_executor.invoke(
    {
        "input": "Tell me the number of messages sent in the #introductions channel from the past month."
    }
)



> Entering new AgentExecutor chain...
First, I need to identify the channel ID for the #introductions channel.
Action: get_channelid_name_dict
Action Input: None[{"id": "C052SCUP4UD", "name": "general", "created": 1681297313, "num_members": 1}, {"id": "C052VBBU4M8", "name": "test-bots", "created": 1681297343, "num_members": 2}, {"id": "C053805TNUR", "name": "random", "created": 1681297313, "num_members": 2}]The #introductions channel is not listed in the observed channels. I need to inform the user that the #introductions channel does not exist or is not accessible.
Final Answer: The #introductions channel does not exist or is not accessible.

> Finished chain.


{'input': 'Tell me the number of messages sent in the #introductions channel from the past month.',
 'output': 'The #introductions channel does not exist or is not accessible.'}

## API reference

For detailed documentation of all __ModuleName__Toolkit features and configurations head to the [API reference](https://api.python.langchain.com/en/latest/agent_toolkits/langchain_community.agent_toolkits.slack.toolkit.SlackToolkit.html).